In [3]:
import scanpy as sc
import functools
import os
import sys
import traceback
from typing import Literal, Optional

import functools
import os
import sys
import traceback
from typing import Literal, Optional

import hydra
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import numpy as np
import optax
import orbax
import scanpy as sc
import wandb
from omegaconf import DictConfig, OmegaConf
from ott.neural import datasets
from ott.neural.methods.flows import dynamics, otfm
from ott.neural.networks.layers import time_encoder
from ott.solvers import utils as solver_utils
from torch.utils.data import DataLoader
from tqdm import tqdm


import hydra
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import numpy as np
import optax
import orbax
import scanpy as sc
import wandb
from omegaconf import DictConfig, OmegaConf
from ott.neural import datasets
from ott.neural.methods.flows import dynamics, otfm
from ott.neural.networks.layers import time_encoder
from ott.solvers import utils as solver_utils
from torch.utils.data import DataLoader
from tqdm import tqdm

from ot_pert.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
from ot_pert.nets.nets import CondVelocityField
from ot_pert.utils import ConditionalLoader

In [4]:
adata = sc.read_h5ad(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_300_{0}.h5ad")
adata.obsm

AxisArrays with keys: X_pca, cell_line_emb, ecfp, ecfp_cell_line, ecfp_cell_line_dose, ecfp_cell_line_dose_more_dose, ecfp_cell_line_logdose, ecfp_cell_line_logdose_more_dose

In [5]:
obsm_key_data = "X_pca"
obsm_key_cond = "ecfp_cell_line_logdose"

In [6]:
def load_data(adata, return_dl: bool = True, batch_size=1024):
    """Loads data and preprocesses it based on configuration."""
    dls = []
    data_source = {}
    data_target = {}
    data_source_decoded = {}
    data_target_decoded = {}
    data_conditions = {}
    for cond in adata.obs["condition"].cat.categories:
        if "Vehicle" not in cond:
            src_str_unique = list(adata[adata.obs["condition"] == cond].obs["cell_type"].unique())
            assert len(src_str_unique) == 1
            src_str = src_str_unique[0] + "_Vehicle_0.0"
            source = adata[adata.obs["condition"] == src_str].obsm[obsm_key_data]
            source_decoded = adata[adata.obs["condition"] == src_str].X.A
            target = adata[adata.obs["condition"] == cond].obsm[obsm_key_data]
            target_decoded = adata[adata.obs["condition"] == cond].X.A
            conds = adata[adata.obs["condition"] == cond].obsm[obsm_key_cond]
            assert np.all(np.all(conds == conds[0], axis=1))
            conds = np.tile(conds[0], (len(source), 1))
            if return_dl:
                dls.append(
                    DataLoader(
                        datasets.OTDataset(
                            datasets.OTData(
                                lin=source,
                                condition=conds,
                            ),
                            datasets.OTData(lin=target),
                        ),
                        batch_size=batch_size,
                        shuffle=True,
                    )
                )
            else:
                data_source[cond] = source
                data_target[cond] = target
                data_source_decoded[cond] = source_decoded
                data_target_decoded[cond] = target_decoded
                data_conditions[cond] = conds
    if return_dl:
        return ConditionalLoader(dls, seed=0)

    deg_dict = {k: v for k, v in adata.uns["rank_genes_groups_cov_all"].items() if k in data_conditions.keys()}

    return {
        "source": data_source,
        "target": data_target,
        "source_decoded": data_source_decoded,
        "target_decoded": data_target_decoded,
        "conditions": data_conditions,
        "deg_dict": deg_dict,
    }


def load_data_2(adata, return_dl: bool = True, batch_size=1024):
    """Loads data and preprocesses it based on configuration."""
    dls = []
    src_str_to_idx = {}
    src_idx_to_data = {}
    tgt_str_to_idx = {}
    tgt_idx_to_data = {}
    cond_str_to_idx = {}    
    cond_idx_to_data = {}
    src_idcs = []
    cond_idcs = []
    tgt_idcs = []
    
    for cond in adata.obs["condition"].cat.categories:
        if "Vehicle" not in cond:
            src_str_unique = list(adata[adata.obs["condition"] == cond].obs["cell_type"].unique())
            assert len(src_str_unique) == 1
            src_str = src_str_unique[0] + "_Vehicle_0.0"
            if src_str in src_str_to_idx:
                src_idx = src_str_to_idx[src_str]
            else:
                src_idx = len(src_str_to_idx)
                src_str_to_idx[src_str] = src_idx
                src_idx_to_data[src_idx] = jnp.asarray(adata[adata.obs["condition"] == src_str].obsm[obsm_key_data])
            if cond in tgt_str_to_idx:
                tgt_idx = tgt_str_to_idx[cond]
            else:
                tgt_idx = len(tgt_str_to_idx)
                tgt_str_to_idx[cond] = tgt_idx
                tgt_idx_to_data[tgt_idx] = jnp.asarray(adata[adata.obs["condition"] == cond].obsm[obsm_key_data])
            if cond in cond_str_to_idx:
                cond_idx = cond_str_to_idx[cond]
            else:
                cond_idx = len(cond_str_to_idx)
                cond_str_to_idx[cond] = cond_idx
                cond_idx_to_data[cond_idx] = jnp.asarray(adata[adata.obs["condition"] == cond].obsm[obsm_key_cond][0])
            
            
            src_idcs.append(src_idx)
            cond_idcs.append(cond_idx)
            tgt_idcs.append(tgt_idx)
            

    if return_dl:
        return JaxSampler(src_idcs, cond_idcs, tgt_idcs, src_str_to_idx, src_idx_to_data, tgt_str_to_idx, tgt_idx_to_data, cond_str_to_idx, cond_idx_to_data, batch_size=batch_size)



In [7]:
from types import MappingProxyType
from typing import Any, Dict, List, Tuple, Union, Literal, Optional
from functools import partial

import jax
import jax.numpy as jnp
from jax import vmap

from ott.solvers.linear import sinkhorn
from ott.geometry.pointcloud import PointCloud

from juniper.types import Tensor


class JaxSampler:
    """Data sampler for Jax."""

    def __init__(
        self,
        idcs_source: jax.Array,
        idcs_conditions: jax.Array,
        idcs_target: jax.Array,
        src_str_to_idx: Dict[str, int], 
        src_idx_to_data: Dict[int, jax.Array], 
        tgt_str_to_idx: Dict[str, int], 
        tgt_idx_to_data: Dict[int, jax.Array], 
        cond_str_to_idx: Dict[str, int], 
        cond_idx_to_data: Dict[int, jax.Array],
        batch_size: int,

    ):
        assert len(idcs_source) == len(idcs_conditions)
        assert len(idcs_source) == len(idcs_target)
        """Initialize data sampler."""
        self.batch_size = batch_size
        self.idcs_source = idcs_source
        self.idcs_conditions = idcs_conditions
        self.idcs_target = idcs_target
        self.n_conditions = len(idcs_source)
        self.src_str_to_idx = src_str_to_idx
        self.src_idx_to_data = src_idx_to_data
        self.tgt_str_to_idx = tgt_str_to_idx
        self.tgt_idx_to_data = tgt_idx_to_data
        self.cond_str_to_idx = cond_str_to_idx
        self.cond_idx_to_data = cond_idx_to_data
        

        @partial(jax.jit, static_argnames=["dist_idx"])
        def _sample_batch(
            dist_idx: jax.Array,
            rng: jax.Array,
        ) -> Dict[str, jax.Array]:
            """Jitted sample function."""
            rng_1, rng_2, rng_3 = jax.random.split(rng, 3)
            src_idx, conds_idx, tgt_idx = self.idcs_source[dist_idx],  self.idcs_conditions[dist_idx],  self.idcs_target[dist_idx]
    
            src = self.src_idx_to_data[src_idx]
            tgt = self.tgt_idx_to_data[tgt_idx]
            cond = np.tile(self.cond_idx_to_data[conds_idx], (len(src), 1))
             
            
            
            source_idcs = jax.random.choice(
                rng_2,
                len(src),
                replace=True,
                shape=[self.batch_size],
            )
    
            tgt_idcs = jax.random.choice(
                rng_3,
                len(tgt),
                replace=True,
                shape=[self.batch_size],
            )
    
            return {"src_lin": src[source_idcs], "tgt_lin": tgt[tgt_idcs], "src_condition": cond}


        self.sample_batch=_sample_batch

    
    

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dl1 = load_data(adata)

2024-07-11 14:28:13.165780: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [8]:
dl2 = load_data_2(adata)

In [10]:
dl2.n_conditions

1199

In [17]:
len(dl2.idcs_conditions)

1199

In [ ]:

cond_str_to_idx: Dict[str, int], 
        cond_idx_to_data: Dict[int, jax.Array],

In [25]:
dl2.src_idx_to_data[0].shape

(2787, 300)

In [26]:
len(dl2.src_idx_to_data)

3

In [27]:
len(dl2.tgt_idx_to_data)

1199

In [163]:

def profile_dl1(num_iters, rng):
    
    for it in range(num_iters):
        rng, rng_resample, rng_step_fn = jax.random.split(rng, 3)
        batch = next(dl1)
        batch = jtu.tree_map(jnp.asarray, batch)
    
        src, tgt = batch["src_lin"], batch["tgt_lin"]
        src_cond = batch.get("src_condition")
    return 1

def profile_dl2(num_iters, rng):
    for it in range(num_iters):
        rng, rng_resample, rng_step_fn = jax.random.split(rng, 3)
        idx = int(jax.random.randint(
                rng, shape=[], minval=0, maxval=dl2.n_conditions
            ))
        batch = dl2.sample_batch(idx, rng)
    
        src, tgt = batch["src_lin"], batch["tgt_lin"]
        src_cond = batch.get("src_condition")
    return 1

In [164]:
import timeit

In [165]:
num_iters=1000

In [166]:
profile_dl1_partial = functools.partial(profile_dl1, num_iters=num_iters, rng=rng)
profile_dl2_partial = functools.partial(profile_dl2, num_iters=num_iters, rng=rng)

In [167]:

num_runs = 1

# Measure the execution time
execution_time = timeit.timeit('profile_dl1_partial()', globals=globals(), number=num_runs)


In [168]:
execution_time

40.13994951173663

In [169]:
num_runs = 1

# Measure the execution time
execution_time = timeit.timeit('profile_dl2_partial()', globals=globals(), number=num_runs)

In [170]:
execution_time

219.64013634249568

In [2]:
num_runs = 1

# Measure the execution time
execution_time = timeit.timeit('profile_dl2_partial()', globals=globals(), number=num_runs)

NameError: name 'timeit' is not defined

In [ ]:
execution_time

In [130]:

num_runs = 5

# Measure the execution time
execution_time = timeit.timeit('profile_dl1_partial()', globals=globals(), number=num_runs)


In [131]:
execution_time 

20.433078983798623

In [150]:
num_runs = 1

# Measure the execution time
execution_time = timeit.timeit('profile_dl2_partial()', globals=globals(), number=num_runs)

In [151]:
execution_time

0.04699268192052841

In [152]:
num_runs = 5

# Measure the execution time
execution_time = timeit.timeit('profile_dl2_partial()', globals=globals(), number=num_runs)

In [153]:
execution_time

0.2392442300915718

In [60]:
execution_time

0.4187095109373331

In [50]:
execution_time

14.171960074454546

In [ ]:
from types import MappingProxyType
from typing import Any, Dict, List, Tuple, Union, Literal, Optional
from functools import partial

import jax
import jax.numpy as jnp
from jax import vmap

from ott.solvers.linear import sinkhorn
from ott.geometry.pointcloud import PointCloud

from juniper.types import Tensor


class JaxSampler:
    """Data sampler for Jax."""

    def __init__(
        self,
        data: List[Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]], # src, src_condition, tgt
        src_str_to_idx, 
        src_idx_to_data, 
        tgt_str_to_idx, 
        tgt_idx_to_data, 
        cond_str_to_idx, 
        cond_idx_to_data,
        batch_size: int,
    ):
        """Initialize data sampler."""
        self.batch_size = batch_size
        self.data = data
        self.n_conditions = len(data)
        self.src_str_to_idx = src_str_to_idx
        self.src_idx_to_data = src_idx_to_data
        self.tgt_str_to_idx = tgt_str_to_idx
        self.tgt_idx_to_data = tgt_idx_to_data
        self.cond_str_to_idx = cond_str_to_idx
        self.cond_idx_to_data = cond_idx_to_data

def _sample_batch(
    rng: jax.Array,
    batch_size: int,
    data: List[Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]],
    src_idx_to_data,
    tgt_idx_to_data,
    cond_idx_to_data,
    n_conditions: int
) -> Dict[str, jnp.ndarray]:
    """Jitted sample function."""
    rng_1, rng_2, rng_3 = jax.random.split(rng, 3)
    # sample condition
    dist_idx = jax.random.choice(rng_1, n_conditions)
    src_idx, conds_idx, tgt_idx = data[dist_idx]

    src = src_idx_to_data[src_idx]
    tgt = tgt_idx_to_data[tgt_idx]
    cond = cond_idx_to_data[conds_idx]

    source_idcs = jax.random.choice(
        rng_2,
        len(src),
        replace=True,
        shape=[batch_size],
    )

    tgt_idcs = jax.random.choice(
        rng_3,
        len(tgt),
        replace=True,
        shape=[batch_size],
    )

    return {"src_lin": src[source_idcs], "tgt_lin": tgt[tgt_idcs], "src_condition": cond}

# Jit the _sample_batch function
sample_batch_jitted = jax.jit(_sample_batch)

# Usage
sampler = JaxSampler(
    data=[...],  # your data here
    src_str_to_idx=,
    src_idx_to_data=...,
    tgt_str_to_idx=...,
    tgt_idx_to_data=...,
    cond_str_to_idx=...,
    cond_idx_to_data=...,
    batch_size=32
)

rng = jax.random.PRNGKey(0)
batch = sample_batch_jitted(
    rng,
    sampler.batch_size,
    sampler.data,
    sampler.src_idx_to_data,
    sampler.tgt_idx_to_data,
    sampler.cond_idx_to_data,
    sampler.n_conditions
)

print(batch)


In [181]:
adata_test_path= "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_biolord_split_30.h5ad"
adata_ood_path="/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_biolord_split_30.h5ad"

In [182]:
adata_test = sc.read_h5ad(adata_test_path)

In [183]:
adata_ood = sc.read_h5ad(adata_ood_path)

In [184]:
adata_ood.uns


OrderedDict()

In [186]:
adata_test.uns.keys()

dict_keys(['hvg', 'log1p', 'pca', 'rank_genes_groups_cov_all'])

In [187]:
adata_ood.uns['rank_genes_groups_cov_all'] = adata_test.uns['rank_genes_groups_cov_all']

In [188]:
adata_ood.write("/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_biolord_split_30.h5ad")

In [190]:
adata_test.varm

AxisArrays with keys: PCs

In [191]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_biolord_split_30.h5ad"
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_biolord_split_30.h5ad"

In [ ]:
adata_train = sc.read_h5ad(adata_train_path)

In [ ]:
adata_train.varm

In [1]:
import scanpy as sc

In [ ]:
adata_train_